In [23]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from order_of_magnitude import order_of_magnitude
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from itertools import combinations

from scipy.stats import mannwhitneyu, wilcoxon, kruskal
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import chi2, chi2_contingency, chisquare

from fitter import Fitter, get_common_distributions, get_distributions
from scipy.stats import kstest, ks_2samp

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, r2_score
import statsmodels.api as sm

from sklearn.decomposition import PCA

In [39]:
data = pd.read_excel('Exercise.xlsx')
data.columns = list(data.iloc[0, :].values)
data.drop(index=0, inplace=True)
data.drop('Name', axis=1, inplace=True)
data.head()

C:\Users\Akshit Tayade\anaconda3\envs\akshit_venv\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Gender,Age,Height(cm),Weight(kg),Lifestyle,Exercise,Daily Calories Intake,Target,How motivated are you for being healthy\n(in %),Body Mass Index (BMI),Basal Metabolic Rate (BMR),Type,Body Fat Percentage (BFP),Body Type,Daily Calorie Need (Based on BMR and Exercise)\n(in Calorie),Total Daily Energy Expenditure (TDEE) (Based on BMR and Lifestyle)\n(in Calorie),Calorie Difference (Daily Calorie Intake - TDEE),Suggestion
1,Male,18,152,50,1. Sedentary or light activity (e.g. - Office ...,1. Sedentary (little or no exercise),2067.152431,Loose Weight,80,21.641274,1392.866,Boy,17.878324,Normal,1671.4392,2131.08498,-0.03,LOW CARB DIET (25:35:40)
2,Male,18,152,54,1. Sedentary or light activity (e.g. - Office ...,2. Lightly active (light exercise/sports 1-3 d...,1705.730935,Maintain Weight,78,23.372576,1447.866,Boy,20.49259,Normal,1990.81575,2215.23498,-0.23,BALANCE DIET (55:25:20)
3,Male,18,152,58,1. Sedentary or light activity (e.g. - Office ...,3. Moderately active (moderate exercise/sports...,1910.142686,Gain Muscle,65,25.103878,1502.866,Boy,23.106856,Overweight,2329.4423,2299.38498,-0.18,BALANCE DIET (55:25:20)
4,Male,18,152,62,3. Vigorously active (Agricultural worker (non...,4. Very active (hard exercise/sports 6-7 days ...,2552.952908,Loose Weight,82,26.83518,1557.866,Boy,25.721122,Overweight,2687.31885,3505.1985,-0.271667,KETOGENIC DIET (5:35:60)
5,Male,18,152,66,1. Sedentary or light activity (e.g. - Office ...,1. Sedentary (little or no exercise),1826.086885,Maintain Weight,75,28.566482,1612.866,Boy,28.335388,Overweight,1935.4392,2467.68498,-0.26,BALANCE DIET (55:25:20)


In [40]:
# Mapping String to numeric values.
data['Gender']=data['Gender'].map({"Male":0,"Female":1})
data['Body Type']=data['Body Type'].map({"Underweight":0,"Normal":1,"Overweight":2,"Obese":3})
data['Type']=data['Type'].map({"Boy":0,"Girl":1,"Adult Male":2,"Adult Female":3})
data['Suggestion']=data['Suggestion'].map({"LOW CARB DIET (25:35:40)":0,"BALANCE DIET (55:25:20)":1,"ZONE DIET (40:30:30)":2,"KETOGENIC DIET (5:35:60)":3,"DEPLETION DIET (DYNAMIC)":4,"HIGH CARB DIET(60:20:20)":5})
data['Target']=data['Target'].map({"Loose Weight":0,"Maintain Weight":1,"Gain Muscle":2})
data['Lifestyle']=data['Lifestyle'].map({"1. Sedentary or light activity (e.g. - Office worker getting little or no exercise)":0,"2. Active or moderately active (Construction worker or person running one hour daily)":1,"3. Vigorously active (Agricultural worker (non mechanized) or person swimming two hours daily)":2})
data['Exercise']=data['Exercise'].map({"1. Sedentary (little or no exercise)":0,"2. Lightly active (light exercise/sports 1-3 days/ week)":1,"3. Moderately active (moderate exercise/sports 3-5days/week)":2,"4. Very active (hard exercise/sports 6-7 days a week)":3})

data = data.apply(pd.to_numeric)

In [41]:
data.drop('Body Type', axis=1, inplace=True)

In [42]:
sc_scaler = StandardScaler()
X_ = sc_scaler.fit_transform(data.iloc[:, :-1])
X_ = pd.DataFrame(X_, columns=list(data.columns)[:-1])
X_

,Gender,Age,Height(cm),Weight(kg),Lifestyle,Exercise,Daily Calories Intake,Target,How motivated are you for being healthy\n(in %),Body Mass Index (BMI),Basal Metabolic Rate (BMR),Type,Body Fat Percentage (BFP),Daily Calorie Need (Based on BMR and Exercise)\n(in Calorie),Total Daily Energy Expenditure (TDEE) (Based on BMR and Lifestyle)\n(in Calorie),Calorie Difference (Daily Calorie Intake - TDEE)
0,-1.0,-1.704309,-1.46385,-1.243933,-1.224988,-1.360019,-0.948266,-1.224745,0.237504,-0.592030,-0.541999,-3.618136,-1.298428,-1.199687,-1.055461,-0.168609
1,-1.0,-1.704309,-1.46385,-1.006993,-1.224988,-0.473050,-1.385131,0.000000,0.043670,-0.342490,-0.321641,-3.618136,-1.017369,-0.530306,-0.926927,-1.307818
2,-1.0,-1.704309,-1.46385,-0.770054,-1.224988,0.413919,-1.138050,1.224745,-1.216250,-0.092950,-0.101283,-3.618136,-0.736309,0.179420,-0.798394,-1.023016
3,-1.0,-1.704309,-1.46385,-0.533114,1.224502,1.300887,-0.361060,-1.224745,0.431338,0.156589,0.119074,-3.618136,-0.455250,0.929492,1.043408,-1.545153
4,-1.0,-1.704309,-1.46385,-0.296174,-1.224988,-1.360019,-1.239652,0.000000,-0.247081,0.406129,0.339432,-3.618136,-0.174191,-0.646370,-0.541326,-1.478699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10075,1.0,1.565182,1.46385,0.296174,1.224502,1.300887,1.246541,0.000000,1.109757,-0.404250,-0.455130,0.904534,0.766252,0.411337,0.550863,1.255402
10076,1.0,1.565182,1.46385,0.533114,1.224502,-1.360019,1.136394,1.224745,1.303591,-0.230196,-0.301873,0.904534,0.922045,-1.048948,0.682325,0.913640
10077,1.0,1.565182,1.46385,0.770054,1.224502,-0.473050,-0.283890,-1.224745,-0.731665,-0.056142,-0.148616,0.904534,1.077837,-0.405851,0.813787,-1.250857
10078,1.0,1.565182,1.46385,1.006993,1.224502,0.413919,0.337951,0.000000,0.915923,0.117912,0.004641,0.904534,1.233630,0.265307,0.945249,-0.510371


In [43]:
x_train, x_test, y_train, y_test = train_test_split(X_, data.iloc[:, -1], test_size=0.2, random_state=1)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((8064, 16), (2016, 16), (8064,), (2016,))

## Applying PCA

In [44]:
pca = PCA(0.95)

x_train_pca = pca.fit_transform(x_train)

principalDf = pd.DataFrame(data = x_train_pca)
principalDf

,0,1,2,3,4,5,6,7,8
0,0.031560,1.758078,0.508862,-0.652512,2.111521,1.432676,0.696095,-1.141067,-0.812370
1,1.841426,-0.717038,0.387675,0.392714,1.906753,0.322675,1.768618,1.105119,-1.779554
2,-0.857017,-1.303735,0.089919,-1.238591,0.745677,-1.853521,1.448843,0.159217,0.009238
3,-1.195309,-2.239957,-0.360441,-0.345498,-0.598713,0.846422,0.166924,-0.394788,-1.172329
4,2.664619,2.292496,-1.899828,-0.846191,-0.374566,-0.930613,0.967771,1.444882,-0.396051
...,...,...,...,...,...,...,...,...,...
8059,2.936472,-0.755961,1.283162,-0.782147,0.054575,0.803975,0.779378,-1.026602,-1.052366
8060,-3.953169,-1.118596,0.025675,1.519916,1.030261,0.704737,-0.258710,0.277805,0.269373
8061,1.643295,0.759260,0.320822,0.288987,-2.272325,-1.693259,0.613262,1.279941,-0.166740
8062,0.491938,-0.596245,0.311265,-0.709489,-0.819633,-2.043755,-1.499836,0.300940,1.404237


In [45]:
x_test_pca = pca.transform(x_test)

principalDf = pd.DataFrame(data = x_test_pca)
principalDf

,0,1,2,3,4,5,6,7,8
0,0.653255,2.988512,-1.105870,-0.531529,0.617153,-0.244636,0.391522,0.743517,-1.095979
1,3.274543,1.162000,-1.632701,1.253229,0.868713,-1.457492,-0.045367,-0.106827,-2.214046
2,2.846956,1.707873,-0.381253,-0.709224,-1.438138,-0.350994,-0.394853,1.225344,0.779774
3,-3.780156,0.164875,1.532373,-1.356799,0.413162,0.738158,-0.325845,1.524339,0.694981
4,1.908336,1.321239,0.485815,0.259375,-1.931783,-0.829879,-0.496881,-0.408316,2.204222
...,...,...,...,...,...,...,...,...,...
2011,-1.783138,-1.359287,1.368019,-1.039206,0.511380,-1.893055,0.845105,0.204733,-1.570009
2012,0.097731,2.832073,0.570817,-1.440424,-0.115279,-0.039297,0.443819,-0.511234,-0.012253
2013,-1.743619,-0.044153,0.210559,0.084556,1.148883,0.353018,-2.087437,-0.825628,-0.007544
2014,-0.670815,0.772865,-1.045080,2.086988,0.824417,0.492852,1.129756,-0.319300,0.114025


## Applying Logistic Regression

In [31]:
log_reg_forward = sm.MNLogit(y_train.values.ravel(), x_train_pca).fit()

Optimization terminated successfully.
         Current function value: 1.009358
         Iterations 8


In [32]:
log_reg_forward.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                      y   No. Observations:                 8064
Model:                        MNLogit   Df Residuals:                     8019
Method:                           MLE   Df Model:                           40
Date:                Sun, 11 Dec 2022   Pseudo R-squ.:                  0.3293
Time:                        00:28:06   Log-Likelihood:                -8139.5
converged:                       True   LL-Null:                       -12136.
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
       y=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.9033      0.025    -35.800      0.000      -0.953      -0.854
x2             0.5803      0.032     18.028      0.000       0.517       0.643
x3            -0.0406      0.035     -1.152      0.249      -0.110       0.028
x4            -0.3416      0.039     -8.689      0.000      -0.419      -0.265
x5             0.7141      0.044     16.170      0.000       0.628       0.801
x6            -0.6279      0.045    -14.074      0.000      -0.715      -0.540
x7             0.4056      0.046      8.741      0.000       0.315       0.497
x8             1.1792      0.049     23.990      0.000       1.083       1.276
x9             0.9070      0.053     16.969      0.000       0.802       1.012
------------------------------------------------------------------------------
       y=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.1633      0.026     -6.345      0.000      -0.214      -0.113
x2             0.2262      0.035      6.456      0.000       0.158       0.295
x3            -0.0891      0.040     -2.244      0.025      -0.167      -0.011
x4            -0.5139      0.045    -11.322      0.000      -0.603      -0.425
x5            -0.0331      0.049     -0.675      0.499      -0.129       0.063
x6            -1.7053      0.057    -30.134      0.000      -1.816      -1.594
x7             1.7455      0.059     29.413      0.000       1.629       1.862
x8             3.5879      0.075     47.638      0.000       3.440       3.736
x9             3.2464      0.075     43.465      0.000       3.100       3.393
------------------------------------------------------------------------------
       y=3       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0139      0.019      0.719      0.472      -0.024       0.052
x2            -0.0561      0.027     -2.111      0.035      -0.108      -0.004
x3             0.0269      0.031      0.873      0.383      -0.033       0.087
x4            -0.0813      0.035     -2.316      0.021      -0.150      -0.012
x5             0.0343      0.038      0.897      0.370      -0.041       0.109
x6            -0.4276      0.041    -10.424      0.000      -0.508      -0.347
x7             0.1826      0.042      4.395      0.000       0.101       0.264
x8             1.3277      0.050     26.457      0.000       1.229       1.426
x9            -0.1899      0.040     -4.801      0.000      -0.267      -0.112
------------------------------------------------------------------------------
       y=4       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.2168      0.029     -7.525      0.000      -0.273      -0.160
x2             0.1905      0.038      4.961      0.000       0.115       0.266
x3

In [33]:
log_reg_forward.aic

16368.931752965553

In [47]:
log_reg_forward.bic

16683.714177529553

## Resampling Methods
    - Using Cross validation

In [46]:
from sklearn.model_selection import cross_validate

lg_resultant = cross_validate(LogisticRegression(multi_class='multinomial', n_jobs=-1), 
                              x_train_pca, 
                              y_train, 
                              cv=5,
                              return_train_score=True)

print("Mean Training Score = ", lg_resultant['train_score'].mean()*100)
print("Mean Testing Score = ", lg_resultant['test_score'].mean()*100)

Mean Training Score =  80.3633485602707
Mean Testing Score =  80.09671727388664


    - Using Bootstrap

In [37]:
train_score = []
test_score = []
n_times = 10

## Performing bootstrapping
for i in range(n_times):
    
    pca = PCA(0.95)
    
    X_pca = pca.fit_transform(X_)
    
    x_train, x_test, y_train, y_test = train_test_split(X_pca, data.iloc[:, -1], test_size=0.2, random_state=42+i)
    
    #choose from different tunable hyper parameters
    clf = LogisticRegression(C=10, multi_class='multinomial', n_jobs=-1).fit(x_train, y_train)
    y_pred = clf.predict(x_test)
 
    #Measuring accuracy on Testing Data
    Accuracy = accuracy_score(y_test, y_pred)
    
    # Storing accuracy values
    test_score.append(Accuracy)
    train_score.append(accuracy_score(y_train, clf.predict(x_train)))
    
################################################
# Result of all bootstrapping trials
print("Mean Training Score = ", np.mean(train_score)*100)
print("Mean Testing Score = ", np.mean(test_score)*100)

Mean Training Score =  80.24305555555556
Mean Testing Score =  80.08928571428571
